In [82]:
import pandas as pd
import numpy as np
import random
import os

Configuracion Inicial

In [83]:
import pandas as pd
import numpy as np
import random
import os

file_path = 'Amazon.csv'
if not os.path.exists(file_path):
    print(f"ERROR: Archivo no encontrado en {file_path}. Asegúrate de subirlo.")
    exit()

# Cargar y hacer una copia para ensuciar
df = pd.read_csv(file_path)
df_dirty = df.copy()

print(f"Dataset original cargado. Filas iniciales: {len(df)}")
print("Iniciando la introducción de 11 tipos de errores...")

Dataset original cargado. Filas iniciales: 100000
Iniciando la introducción de 11 tipos de errores...


Introducción de Datos Faltantes (NaN)

In [84]:
# UnitPrice: 5% NaN
mask_unitprice = np.random.choice([True, False], size=df_dirty.shape[0], p=[0.05, 0.95])
df_dirty.loc[mask_unitprice, 'UnitPrice'] = np.nan

# Discount: 3% NaN
mask_discount = np.random.choice([True, False], size=df_dirty.shape[0], p=[0.03, 0.97])
df_dirty.loc[mask_discount, 'Discount'] = np.nan

# City: 1% NaN
mask_city = np.random.choice([True, False], size=df_dirty.shape[0], p=[0.01, 0.99])
df_dirty.loc[mask_city, 'City'] = np.nan
print("Errores de Datos Faltantes (NaN) introducidos en UnitPrice, Discount y City.")

Errores de Datos Faltantes (NaN) introducidos en UnitPrice, Discount y City.


Introducción de Filas Duplicadas

In [85]:
# Añadir 50 duplicados exactos de las primeras 50 filas
duplicates = df_dirty.iloc[:50].copy()
df_dirty = pd.concat([df_dirty, duplicates], ignore_index=True)
print(f"50 filas duplicadas exactas añadidas. Total de filas: {len(df_dirty)}")

50 filas duplicadas exactas añadidas. Total de filas: 100050


Introducción de Valores Atípicos (Outliers)

In [86]:
# Insertar valores extremos en 'Quantity' (0.1% de las filas originales)
outlier_count = int(df.shape[0] * 0.001)
outlier_indices = np.random.choice(df_dirty.index, size=outlier_count, replace=False)
df_dirty.loc[outlier_indices, 'Quantity'] = np.random.choice([999, 1000, 10000], size=outlier_count)
print(f"Valores atípicos extremos introducidos en Quantity ({outlier_count} filas).")

Valores atípicos extremos introducidos en Quantity (100 filas).


Inconsistencias de Formato (Fecha y Símbolo)

In [87]:
# OrderDate: Cambiar 10% a 'MM/DD/YYYY'
date_format_mask = np.random.choice([True, False], size=df_dirty.shape[0], p=[0.10, 0.90])
dates_to_change = pd.to_datetime(df_dirty.loc[date_format_mask, 'OrderDate'], errors='coerce')
df_dirty.loc[date_format_mask, 'OrderDate'] = dates_to_change.dt.strftime('%m/%d/%Y')
print("Inconsistencias de formato de fecha (MM/DD/YYYY) introducidas.")

# TotalAmount: Añadir '€' y convertir a string (Símbolo Extra)
total_amount_mask = np.random.choice([True, False], size=df_dirty.shape[0], p=[0.02, 0.98])

# CORRECCIÓN para evitar FutureWarning: Convertir toda la columna a 'object'
df_dirty['TotalAmount'] = df_dirty['TotalAmount'].astype(object) 

# Ahora introducir el error (string + símbolo) en el 2% de los valores
df_dirty.loc[total_amount_mask, 'TotalAmount'] = df_dirty.loc[total_amount_mask, 'TotalAmount'].astype(str) + '€'
print("Símbolo '€' y conversión a string introducidos en TotalAmount.")

Inconsistencias de formato de fecha (MM/DD/YYYY) introducidas.
Símbolo '€' y conversión a string introducidos en TotalAmount.


Errores Tipográficos y Categorías Extra

In [88]:
# Category (Error 8: Errores de Deletreo)
replacement_map = {'Books': 'Boks', 'Clothing': 'Clothng', 'Electronics': 'Electrincs'}
category_mask = df_dirty['Category'].isin(replacement_map.keys())
category_corrupt_indices = df_dirty[category_mask].sample(frac=0.1, random_state=42).index

for original, corrupted in replacement_map.items():
    df_dirty.loc[category_corrupt_indices, 'Category'] = df_dirty.loc[category_corrupt_indices, 'Category'].replace(original, corrupted)
print("Errores tipográficos introducidos en la columna Category.")

# PaymentMethod (Error 9: Categoría 'UNKNOWN')
payment_mask = np.random.choice([True, False], size=df_dirty.shape[0], p=[0.01, 0.99])
df_dirty.loc[payment_mask, 'PaymentMethod'] = 'UNKNOWN'
print("Categoría 'UNKNOWN' introducida en PaymentMethod.")

Errores tipográficos introducidos en la columna Category.
Categoría 'UNKNOWN' introducida en PaymentMethod.


Tipo de Dato Incorrecto y Encabezado Incorrecto

In [89]:
# ShippingCost
shipping_cost_mask = np.random.choice([True, False], size=df_dirty.shape[0], p=[0.05, 0.95])

# Convertir toda la columna a 'object' (string/mixto) para evitar la FutureWarning
df_dirty['ShippingCost'] = df_dirty['ShippingCost'].astype(object)

# Ahora introduce el string en el 5% de los valores sin generar el Warning
df_dirty.loc[shipping_cost_mask, 'ShippingCost'] = \
    df_dirty.loc[shipping_cost_mask, 'ShippingCost'].apply(lambda x: str(x))

print("Tipo de dato incorrecto (string) introducido en ShippingCost (ahora es tipo 'object').")

Tipo de dato incorrecto (string) introducido en ShippingCost (ahora es tipo 'object').


Guardar y Verificación Final

In [90]:
dirty_file_name = "Amazon_DIRTY.csv"
df_dirty.to_csv(dirty_file_name, index=False, encoding='utf-8')

print("\n--- RESUMEN DEL DATASET SUCIO ---")
print(f"¡Conjunto de datos sucio guardado como: {dirty_file_name}!")
print("\nInformación de Tipos de Datos (Dtypes):")
df_dirty.info()
print("\n--- PROCESO COMPLETADO ---")


--- RESUMEN DEL DATASET SUCIO ---
¡Conjunto de datos sucio guardado como: Amazon_DIRTY.csv!

Información de Tipos de Datos (Dtypes):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100050 entries, 0 to 100049
Data columns (total 20 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   OrderID        100050 non-null  object 
 1   OrderDate      100050 non-null  object 
 2   CustomerID     100050 non-null  object 
 3   CustomerName   100050 non-null  object 
 4   ProductID      100050 non-null  object 
 5   ProductName    100050 non-null  object 
 6   Category       100050 non-null  object 
 7   Brand          100050 non-null  object 
 8   Quantity       100050 non-null  int64  
 9   UnitPrice      95053 non-null   float64
 10  Discount       97009 non-null   float64
 11  Tax            100050 non-null  float64
 12  ShippingCost   100050 non-null  object 
 13  TotalAmount    100050 non-null  object 
 14  PaymentMethod  100050 non-null